<img src="figures/hop-logo-transparent-bunny.png" style="float: right; height: 5em; padding-top: 1em;">

# Hop.SCIMMA Tutorial for the 2021 LSST Brokers Workshop

*Streaming hub and platform for multi-messenger astronomy.*

## Welcome to Hop.SCIMMA!

Congratulations! The fact you're here means you were successful in logging in using your institutional credentials. This means one less password to remember and worry about, and the ability for your administrators to help manage your account.

## A Very Brief Demo focusing on IAM

In this short demo, we will show the services available on Hop.SCIMMA, **with a special focus on IAM integration**. We note the interfaces we show will mostly be used by tool builders and power user; most *astronomers* will be using these services through interfaces such as [TOMs](https://lco.global/tomtoolkit/) or systems like [TreasureMap](http://treasuremap.space/). Other demos and talks in this conference will discuss those higher-level integrations.

### Step 1: Create new credentials (an authentication token)

Please go to https://my.hop.scimma.org and create new credentials. **Don't close the page once you do!**

Then open a terminal, run the following command:
```sh
$ hop auth add
```
and follow the instructions on how to copy/paste the generated credentials.

<font color=red>**NOTE: It can take up to 3 minutes for the credentials to become active.**</font> We're working on reducing this activation period (to zero).

### Step 2: Connectivity test

The snippet of code below uses [hop-client](https://hop-client.readthedocs.io/en/stable/index.html) to connect to our Kafka's `sys.heartbeat` stream and print the first five messages received:

In [ ]:
from hop import Stream

with Stream(persist=True).open("kafka://kafka.scimma.org/sys.heartbeat", "r") as src:
    for i, message in zip(range(5), src):
        print(i, message)

### Step 3: Users can create their own topics

<font color=red>**NOTE: You can currently do this step only if you've been alowed to by an administrator**</font>

If you're an owner of a group ("organization") in hop.SCIMMA, you're allowed to create and manage topics within that group.

### Step 4: Write and run a simple stream processor

The code below reads from the `sys.heartbeat` topic, converts the timestamp to a human-readable format, and writes to my newly created `demo.heartbeat` topic.

In [ ]:
from datetime import timezone, datetime
from hop import Stream, stream

with Stream(persist=True).open("kafka://kafka.scimma.org/sys.heartbeat", "r") as src, \
    stream.open("kafka://kafka.scimma.org/demo.heartbeat", "w") as dest:

    for i, message in zip(range(5), src):
        t = datetime.fromtimestamp(message["timestamp"]/1e6, tz=timezone.utc)
        message['iso'] = t.isoformat()

        print(i, message)
        dest.write(message)

The stream processor above is run from the Jupyter notebook just for demo purposes; our preferred way to run this type of code is as a Kubernetes pod.

### Step 5a: Try consuming from `demo.heartbeat`

In [ ]:
from hop import Stream
from hop.io import StartPosition

with Stream(persist=True).open("kafka://kafka.scimma.org/demo.heartbeat", "r") as src:
    for i, message in zip(range(5), src):
        print(i, message)

### Step 5b: Try consuming from `demo.private`

This will fail as you're not a member of the `demo` group.

In [ ]:
from hop import Stream
from hop.io import StartPosition

with Stream(persist=True).open("kafka://kafka.scimma.org/demo.private", "r") as src:
    for i, message in zip(range(5), src):
        print(i, message)

## Summary

We've given you a glimpse of how Hop.SCIMMA (and Hopskotch) can be used to enable creation of ad-hoc collaborations and topics with fine-grained permissions. The goal is to ultimately provide a hub for MMA events, a toolkit for local deployments, and a hosted service which collaborations and individuals can use to build and deploy powerful streaming applications.

Examples of early usage are the [SNEWS 2.0 system](https://snews.bnl.gov/), and the upcoming integration with the [TOM toolkit](https://lco.global/tomtoolkit/) and [TreasureMap](http://treasuremap.space/). See demos and talks in this workshop by the Las Cumbres team.

## Potential uses within the LSST ecosystem

What can Hop.SCIMMA offer to the broader LSST broker ecosystem?

**Technical opportunities:**
* The [infrastructure underlying Hop.SCIMMA](https://github.com/scimma) is largely written in Python, and aimed to be reusable and transferrable to other projects. It may be possible to generalize SCIMMA's IAM ("Identity and Access Management") modules for use within other services (and community development).
* You could also use the full Hopskotch system as a basis for your platform.

**Operational opportunities:**
* Hop.SCIMMA is being considered by LIGO and similar experiments as an end-point to which to transmit machine-readable discoveries. Brokers in the LSST ecosystem can connect to Hop.SCIMMA to receive these messages.
* Hop.SCIMMA could be listened to by Rubin observatory itself, for TOO-triggering alerts.

**User opportunities:**
* Hop.SCIMMA services could be used to write simple (and ultimately more complex) stream transformation filters.
* Hop.SCIMMA could be used to wire together your experiments and observatories, create ad-hoc collaborations, etc.

## Background: About SCIMMA, Hopskotch, and Hop.SCIMMA

<img src="figures/scimma-hub.png" style="float: right; height: 30em; padding-top: 5em;">

[SCIMMA](https://scimma.org/) is an NSF-funded, multi-institutional project to understand the needs for and develop cyberinfrastructure in support of multi-messenger astronomy (MMA).

One such need is the ability to reliably and with low latency connect MMA experiments, follow-up facilities, and scientists. This has led to development of [Hopskotch](https://scimma.org/projects.html), **a clustered deployment of Apache Kafka strongly integrated with IAM systems common in research space (namely, [InCommon](https://www.incommon.org/) and [CILogon](https://www.cilogon.org/home))**. Hopskotch can be used as a **toolkit to very quickly bootstrap low-level streaming services in your organization**.

SCIMMA also operates [Hop.SCIMMA](https://hop.scimma.org) a deployment of Hopskotch-as-a-service that will be made available for MMA related projects and science. The goals of Hop.SCIMMA include:
1. Provide a hub for MMA alert producers onto which they can publish their discoveries with very small latency (10s of ms), high reliability (no message loss), fine-grained permissions, and ease of use.
1. Provide a hub to which MMA alert consumers (follow-up observatories, groups, individual scientists) can connect to receive MMA alerts, as well as publish enriched or transformed alerts.
1. Provide a reliable, performant, easy to use, platform (PaaS) which projects and individuals can use to exchange messages and connect experiments.

<hr>

<img src="figures/nsf1.jpg" style="float: right; height: 5em">

This project is supported by National Science Foundation grants OAC-1841625, OAC-1934752. Any opinions, findings, conclusions or recommendations expressed in this material are those of the developers and do not necessarily reflect the views of the National Science Foundation.